1. Numpy is fast bcs. it uses Compiled code behind the scenes

We can see the benefit of this by trying Numba, a Python compiler and one of the simplest ways to speed up your code 100x

In [ ]:
import numpy as np


my_array = np.random.randint(low=0, high=1000, size=(1000000000))

In [ ]:
np.info(my_array)

class:  ndarray
shape:  (1000000000,)
strides:  (8,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x280000000
byteorder:  little
byteswap:  False
type: int64


The above array is about 8GB (1B int64 elements)

### Let's see how long it takes Numpy to sum these

In [ ]:
%%timeit -n 3

np.sum(my_array)

164 ms ± 4.52 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [ ]:
%%timeit -r 2

my_sum = 0

for i in range(len(my_array)):
    my_sum += my_array[i]

1min 8s ± 889 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


Similarly, using the `sum()` function

In [ ]:
%%timeit -r 2

sum(my_array)

33.5 s ± 24.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


### What do I do when I need to do something Numpy doesn't offer! Python is so slow :(


Do not despair! We can manually do what Numpy is doing, to try matching its speed


You may discount compilation, thinking: it's the same code, how much can that speed up. Well, let's see


Below is literally the same loop code as above, just in its own function and with the @jit thing on top, called a **decorator**

In [ ]:
from numba import jit

@jit(nopython=True)
def fast_sum(my_array):
    my_sum = 0

    for i in range(len(my_array)):
        my_sum += my_array[i]

    return my_sum

In [ ]:
%%timeit

fast_sum(my_array)

160 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Not bad, eh? Literally went from 33 seconds to .4, ~100x faster, matching the speed of Python.

Compilation is amazing, and so Numba is amazing! It's the second thing I recommend to people to speed up their Python, right after thinking about designing a better Algorithm or using a more suitable Data Structure

### <font color="orange"> But we can go further!</font>

In [ ]:
from numba import jit

@jit(nopython=True, fastmath=True)
def faster_sum(my_array):
    my_sum = 0

    for i in range(len(my_array)):
        my_sum += my_array[i]

    return my_sum

In [ ]:
%%timeit

faster_sum(my_array)

159 ms ± 798 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from numba import jit, void, int64

@jit(void(int64[:]))
def fasterer_sum(my_array):
    my_sum = 0

    for i in range(len(my_array)):
        my_sum += my_array[i]

    return my_sum

/var/folders/21/rybqmdj15yz5lp1lppq1761m0000gn/T/ipykernel_31590/2542973380.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fasterer_sum" failed type inference due to: No conversion from int64 to none for '$38return_value.1', defined at None

File "../../../var/folders/21/rybqmdj15yz5lp1lppq1761m0000gn/T/ipykernel_31590/2542973380.py", line 10:
<source missing, REPL/exec in use?>

During: typing of assignment at /var/folders/21/rybqmdj15yz5lp1lppq1761m0000gn/T/ipykernel_31590/2542973380.py (10)

File "../../../var/folders/21/rybqmdj15yz5lp1lppq1761m0000gn/T/ipykernel_31590/2542973380.py", line 10:
<source missing, REPL/exec in use?>

  @jit(void(int64[:]))
/var/folders/21/rybqmdj15yz5lp1lppq1761m0000gn/T/ipykernel_31590/2542973380.py:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "fasterer_sum" failed type inference due to: Cannot determine Numba type of <class 'numba.c

In [ ]:
%%timeit

fasterer_sum(my_array)

160 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from numba import njit, prange # parallel range

@njit(parallel=True)
def parallel_fast_sum(my_array):
    my_sum = 0

    for i in prange(len(my_array)):
        my_sum += my_array[i]

    return my_sum

In [ ]:
%%timeit

parallel_fast_sum(my_array)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


68.9 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As we can see, not only can we match Numpy's performance, we can even exceed it without much effort!

References:

https://numba.readthedocs.io/en/stable/index.html

https://numba.readthedocs.io/en/stable/user/parallel.html

https://numba.readthedocs.io/en/stable/user/jit.html

https://numpy.org/doc/stable/reference/simd/index.html?highlight=cpu

## A more difficult example - the RBF Kernel

In [ ]:
from sklearn import datasets

iris_data = datasets.load_iris().data
iris_label = datasets.load_iris().target

In [ ]:
iris_data[:5,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

Creating H from the RBF Kernel

In [ ]:
def generate_H_slow(data, spread):
    # Double for-loop (slow) version
    # data - label-free dataset
    # spread - sigma, standard deviation
    n = data.shape[0]
    H = np.zeros((n,n))
    for j in range(n):
        W = data[j,:]
        for i in range(n):
            H[i, j] = np.exp(-((data[i,:]-W) @ (data[i,:]-
                              W).T)/(2*spread**2))
    return H

In [ ]:
%%timeit
generate_H_slow(iris_data, 1)

42.3 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### The unoptimized version took 42ms for Iris. Not great, not terrible

In [ ]:
def generate_H_fast(data, spread):
    n = data.shape[0]
    H = np.zeros((n,n))
    for j in range(n):
        # First, let's convert this to a Matrix subtraction
        #   1. We subtract W from a different row of data in each step. Let's instead subtract W from all data
        W = data[j,:]
        # data - W should be equivalent to data.iloc[i,:]-W, for all i bcs. of Numpy broadcasting -https://numpy.org/doc/stable/reference/generated/numpy.broadcast.html#numpy.broadcast
        H[:, j] = np.exp(-np.linalg.norm(data-W, ord=2, axis=1)**2/(2*spread**2))
    return H

In [ ]:
%%timeit
generate_H_fast(iris_data, 1)

1.14 ms ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Much better, 40x faster

But it actually took quite a bit of work and some redesigning. Nobody has time for that!

In [ ]:
@jit(nopython=True)
def generate_H_numba(data, spread):
    # Double for-loop (slow) version
    # data - label-free dataset
    # spread - sigma, standard deviation
    n = data.shape[0]
    H = np.zeros((n,n))
    for j in range(n):
        W = data[j,:]
        for i in range(n):
            H[i, j] = np.exp(-((data[i,:]-W) @ (data[i,:]-
                              W).T)/(2*spread**2))
    return H

In [ ]:
%%timeit
generate_H_numba(iris_data, 1)

1.91 ms ± 9.89 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


About half as slow as Numpy, but still 20x better than the original!